In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import requests
import re
from bs4 import BeautifulSoup

In [14]:
class pipeline():
    def __init__(self, medium_content_url, module_url):
        assert type(medium_content_url) == str
        assert type(module_url) == str
        self.medium_content_url = medium_content_url
        self.module_url = module_url
        self.prepare_text_from_html(self.medium_content_url)
        self.loading_module(self.module_url)
        self.text_embedded = self.run_embedding(self.embed, self.text)
    
    def ask_question_from_text(self, question):
        self.question = list(question)
        self.question_embedded = self.run_embedding(self.embed, self.question)
    
    # function for downloading content from the web
    def content_downloader(self, url):
        request = requests.get(url)
        soup = BeautifulSoup(request.content, "lxml") 
        self.html = str(soup.find("div", {"class": "postArticle-content js-postField js-notesSource js-trackedPost"}))
    
    # cleaning the content and return sentences it's not compelete and need to be modified
    def cleaning_text(self, html_part):
        text = re.sub('<[A-Za-z\/][^>]*>', ' ', str(html_part))
        text = re.split(r'\s{2,}', text)[1:-1]
        text = [sentence.replace('\xa0', ' ') for sentence in text]
        text = [sentence.replace('\u200a—\u200a', ' ') for sentence in text]
        return text
    
    # preparing a text for execute caculations on it
    def prepare_text_from_html(self, url):
        self.content_downloader(url)
        self.text = self.cleaning_text(self.html)
    
    # function for loading diffrenet module
    def loading_module(self, module_url):
        # Import the Universal Sentence Encoder's TF Hub module
        embed_object = hub.Module(module_url)
        self.embed_object = embed_object
        
    # function for runinng embedding module on text
    def run_embedding(embed_object, text):
        # Reduce logging output.
        tf.logging.set_verbosity(tf.logging.ERROR)

        with tf.Session() as session:
            session.run([tf.global_variables_initializer(), tf.tables_initializer()])
            message_embeddings = session.run(embed_object(text))

        return message_embeddings